In [60]:
# Chargement des bibliothèques nécessaires
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, Dropout


# Chargement des données et des modèles
with open("data.pkl", "rb") as f:
    questions_matrix, data, best_num_clusters, tfidf_vectorizer, tfidf_vectorizer1 = pickle.load(f)
    
with open("model.pkl", "rb") as f1:
    model, history = pickle.load(f1)


In [61]:
tfidf_matrix = tfidf_vectorizer1.fit_transform(data['body_x'])
tfidf_matrix = tfidf_matrix.toarray()

labels = data['cluster']

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, test_size=0.2, random_state=42)

X_train = X_train.reshape(X_train.shape[0],1, X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1, X_test.shape[1])


In [62]:
lstm = Sequential()
lstm.add(LSTM(units=128, input_shape=(1, X_train.shape[2])))
lstm.add(Dropout(0.5))
lstm.add(Dense(units=best_num_clusters, activation='softmax'))

lstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [63]:
lstm.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 127s 116ms/step - accuracy: 0.6319 - loss: 1.1393
Epoch 2/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 149s 123ms/step - accuracy: 0.9740 - loss: 0.1044
Epoch 3/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 136s 116ms/step - accuracy: 0.9926 - loss: 0.0378
Epoch 4/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 1569s 2s/step - accuracy: 0.9970 - loss: 0.0188
Epoch 5/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 129s 126ms/step - accuracy: 0.9984 - loss: 0.0100
Epoch 6/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 147s 144ms/step - accuracy: 0.9998 - loss: 0.0048
Epoch 7/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 140s 138ms/step - accuracy: 0.9998 - loss: 0.0031
Epoch 8/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 141s 138ms/step - accuracy: 0.9999 - loss: 0.0018


In [64]:
loss, accuracy = lstm.evaluate(X_test, y_test)
print("Perte sur l'ensemble de test:", loss)
print("Précision sur l'ensemble de test:", accuracy)

254/254 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9768 - loss: 0.0911
Perte sur l'ensemble de test: 0.0872756689786911
Précision sur l'ensemble de test: 0.9763692021369934


In [65]:
# Prédire les étiquettes sur l'ensemble de test
predictions = lstm.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, predicted_labels)

# Calculer le F-score
f1 = f1_score(y_test, predicted_labels, average='weighted')

print("Accuracy sur l'ensemble de test:", accuracy)
print("F-score sur l'ensemble de test:", f1)

254/254 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step
Accuracy sur l'ensemble de test: 0.9763692307692308
F-score sur l'ensemble de test: 0.9763640445512691


### BLSTM

In [66]:
blstm = Sequential()
blstm.add(Bidirectional(LSTM(units=128, input_shape=(1, X_train.shape[2]))))
blstm.add(Dropout(0.5))
blstm.add(Dense(units=best_num_clusters, activation='softmax'))

blstm.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [67]:
blstm.fit(X_train, y_train, epochs=8, batch_size=32)

Epoch 1/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 10888s 11s/step - accuracy: 0.6718 - loss: 1.0052
Epoch 2/8
1016/1016 ━━━━━━━━━━━━━━━━━━━━ 279s 216ms/step - accuracy: 0.9838 - loss: 0.0695
Epoch 3/8


In [ ]:
loss, accuracy = blstm.evaluate(X_test, y_test)
print("Perte sur l'ensemble de test:", loss)

# Prédire les étiquettes sur l'ensemble de test
predictions = blstm.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, predicted_labels)

# Calculer le F-score
f1 = f1_score(y_test, predicted_labels, average='weighted')

print("Accuracy sur l'ensemble de test:", accuracy)
print("F-score sur l'ensemble de test:", f1)

### Evaluation du modèle MLP 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Perte sur l'ensemble de test:", loss)

# Prédire les étiquettes sur l'ensemble de test
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, predicted_labels)

# Calculer le F-score
f1 = f1_score(y_test, predicted_labels, average='weighted')

print("Accuracy sur l'ensemble de test:", accuracy)
print("F-score sur l'ensemble de test:", f1)